## Run The Circuit on a Quantum Provider

#### Build The time dependent Hamiltonian

Imports and Routines

In [1]:
import qiskit
import numpy as np
from qiskit.quantum_info import SparsePauliOp

from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from typing import Dict



def get_hamiltonian(t_onebody:Dict, n_sites:int):

    # List of Hamiltonian terms as 3-tuples containing
    # (1) the Pauli string,
    # (2) the qubit indices corresponding to the Pauli string,
    # (3) the coefficient.
    XX_tuples=[]
    YY_tuples=[]
    Z_tuples=[]
    I_tuples=[]
    for (i,j),t_val in t_onebody.items():
    
        if i==j:
            Z_tuples.append(("Z", [i], -0.5*t_val))
            I_tuples.append(('I',[i],0.5*t_val)) 
        else:
            XX_tuples.append( ("XX", [i, j], 0.5*t_val) )
            YY_tuples.append(("YY", [i, j], 0.5*t_val) )
            
    # We create the Hamiltonian as a SparsePauliOp, via the method
    # `from_sparse_list`, and multiply by the interaction term.
    hamiltonian = SparsePauliOp.from_sparse_list([*XX_tuples, *YY_tuples,*Z_tuples,*I_tuples], num_qubits=n_sites)
    return hamiltonian.simplify()


Loading the parameters of $H_Q$

In [2]:
data_onebody=np.load('data/matrix_elements_h_eff_2body/one_body_nn_p.npz')
keys=data_onebody['keys']
values=data_onebody['values']
n_qubits=3

t_onebody={}

for a,key in enumerate(keys):
    i,j=key
    t_onebody[(i,j)]=values[a]
    
print(t_onebody)

t_onebody.pop((0,1))
t_onebody.pop((0,2))
t_onebody.pop((1,2))

print(t_onebody)

{(0, 0): -26.722099999999998, (0, 1): 1.0430999999999995, (0, 2): -3.763858685577873, (1, 0): 1.0430999999999995, (1, 1): -26.722099999999998, (1, 2): 3.763858685577873, (2, 0): -3.763858685577873, (2, 1): 3.763858685577873, (2, 2): -23.4103}
{(0, 0): -26.722099999999998, (1, 0): 1.0430999999999995, (1, 1): -26.722099999999998, (2, 0): -3.763858685577873, (2, 1): 3.763858685577873, (2, 2): -23.4103}


Compute $H_Q$

In [3]:
hamiltonian_q=get_hamiltonian(t_onebody=t_onebody,n_sites=n_qubits)

Compute $H_D$

In [4]:
coupling_term=-26.7221
Z_tuples=[("Z", [0], -0.5*coupling_term)]
I_tuples=[('I',[0],0.5*coupling_term)]

        
# We create the Hamiltonian as a SparsePauliOp, via the method
# `from_sparse_list`, and multiply by the interaction term.
hamiltonian_driver = SparsePauliOp.from_sparse_list([*Z_tuples,*I_tuples], num_qubits=n_qubits)


#### Build the circuit

In [30]:
from qiskit.quantum_info import SparsePauliOp
from qiskit.synthesis import SuzukiTrotter,QDrift,LieTrotter
from qiskit.circuit.library import PauliEvolutionGate
from qiskit import transpile,qpy,qasm2,qasm3
from qiskit.quantum_info import Statevector
from mitiq import zne
#final_state=Statevector(circuit_opt)
time_steps=100
tf=10
time=np.linspace(0,tf,time_steps)
dt=tf/time_steps
#b=0.8
#h=1-(1+b)*(time/tf)+b*(time/tf)**2
h=1-time/tf
circuit_time_evolution=QuantumCircuit(hamiltonian_q.num_qubits,hamiltonian_q.num_qubits)
circuit_time_evolution.x([0]) # initial state

energies_qiskit=np.zeros((time_steps))


for n,t in enumerate(time):
    hamiltonian_t=((h[n])*hamiltonian_driver+(1-h[n])*hamiltonian_q)
    exp_H_t=PauliEvolutionGate(hamiltonian_t,time=dt,synthesis=LieTrotter())
    circuit_time_evolution.append(exp_H_t,range(hamiltonian_q.num_qubits))
    single_particle_vector=np.zeros(2**n_qubits,dtype=np.complex128)
    psi_for_fidelity=np.zeros(n_qubits,dtype=np.complex128)
    # for a in range(n_qubits):
    #     a_mb=2**(a)
    #     final_state=Statevector(circuit_time_evolution).data
    #     single_particle_vector[a_mb]=final_state[a_mb]
    #     psi_for_fidelity[a]=final_state[a_mb]
    # energies_qiskit[n]=Statevector(single_particle_vector).expectation_value(hamiltonian_t)
print(Statevector(circuit_time_evolution).expectation_value(hamiltonian_q))
transpiled_circuit_time_evolution=transpile(circuit_time_evolution.decompose(), optimization_level=3,basis_gates=['cx','h','rz','x'])

(-31.334163248207602+0j)


In [31]:
from mitiq.interface import mitiq_qiskit,mitiq_qibo

qc_qibo=mitiq_qibo.to_qibo(mitiq_qiskit.from_qiskit(transpiled_circuit_time_evolution))
with open("data/qiskit_qa_circuit.qpy", "wb") as f:
    qpy.dump(transpiled_circuit_time_evolution,f)

In [ ]:
with open("data/qa_circuit.qasm", "w") as f:
    f.write(qc_qibo.to_dict())


In [ ]:
transpiled_circuit_time_evolution.draw()

#### Perform Measurement Shots using an IBM clean backend

In [7]:
z_measure_QA_circuit=transpiled_circuit_time_evolution.copy()
z_measure_QA_circuit.measure([0, 1, 2], [0, 1, 2])
z_measure_QA_circuit.measure_all()



#### Classical Backend

In [34]:
print(hamiltonian_q.to_list())

[('IXX', (0.5215499999999997+0j)), ('XIX', (-1.8819293427889365+0j)), ('XXI', (1.8819293427889365+0j)), ('IYY', (0.5215499999999997+0j)), ('YIY', (-1.8819293427889365+0j)), ('YYI', (1.8819293427889365+0j)), ('IIZ', (13.361049999999999+0j)), ('IZI', (13.361049999999999+0j)), ('ZII', (11.70515+0j)), ('III', (-38.42725+0j))]


In [96]:
from qiskit.providers.fake_provider import GenericBackendV2,FakeBackend,FakeQasmBackend,FakeOpenPulse3Q,Fake7QPulseV1,FakeBackend,Fake5QV1,Fake20QV1

from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_aer.noise import NoiseModel
from qiskit_aer import AerSimulator
from qiskit.opflow import PauliExpectation, CircuitSampler, StateFn
# Replace with your actual API token
# service = IBMProvider(token='91830ee51d4a39179a63363dd51d2b588653b869a4cab814c29deb5ac3ded710d02c63902e330e53bbfb36994a605f76f6822b3e608a78b38cc8927e023b0ea9')
# backends = service.backends()
# for backend in backends:
#     print(backend.name)
    
backend=Fake20QV1()

noise_model = NoiseModel.from_backend(backend)

backend=AerSimulator(noise_model=noise_model,n_qubits=3)


hamiltonian_q=get_hamiltonian(t_onebody=t_onebody,n_sites=3)


# Use the pass manager with the layout
# pm = generate_preset_pass_manager(
#     backend=backend,
#     optimization_level=0,
# )


# transpiled_qc=pm.run(transpiled_circuit_time_evolution)
# layout = transpiled_qc.layout

# # h_squared=(hamiltonian_q @ hamiltonian_q).simplify()
# print(hamiltonian_q.to_list())
# estimator=Estimator()
observable = SparsePauliOp.from_list(hamiltonian_q.to_list())

# Create expectation value sampler
sampler = CircuitSampler(backend).convert(StateFn(observable, is_measurement=True) @ StateFn(transpiled))

# Evaluate
expectation_value = sampler.eval().real
print("⟨H⟩ =", expectation_value)

ModuleNotFoundError: No module named 'qiskit.opflow'

In [93]:
import matplotlib.pyplot as plt 
from qiskit.visualization import plot_histogram
%matplotlib inline

print(result.values)


[-9.02389274e-13]


#### Doing the Same Thing using qibo

In [99]:
from mitiq.interface.mitiq_qiskit import from_qiskit
from mitiq.interface.mitiq_qibo import to_qibo




ModuleNotFoundError: No module named 'ply'

In [97]:
qibo_circuit=to_qibo(from_qiskit(transpiled_circuit_time_evolution))

qibo_circuit.draw()

NameError: name 'to_qibo' is not defined

In [18]:
import qiskit
print(qiskit.__version__)

1.4.3


#### Run the Quantum Circuit at different times

In [ ]:

time_steps=100
tf=10
time=np.linspace(0,tf,time_steps)
dt=tf/time_steps

energies_clean=[]
for idx_t in range(time.shape[0]):

    #b=0.8
    #h=1-(1+b)*(time/tf)+b*(time/tf)**2
    h=1-time/tf
    circuit_time_evolution=QuantumCircuit(hamiltonian_q.num_qubits,hamiltonian_q.num_qubits)
    circuit_time_evolution.x([0]) # initial state

    energies_qiskit=np.zeros((time_steps))


    for n,t in enumerate(time[:idx_t]):
        hamiltonian_t=((h[n])*hamiltonian_driver+(1-h[n])*hamiltonian_q)
        exp_H_t=PauliEvolutionGate(hamiltonian_t,time=dt,synthesis=SuzukiTrotter(order=1))
        circuit_time_evolution.append(exp_H_t,range(hamiltonian_q.num_qubits))
        single_particle_vector=np.zeros(2**n_qubits,dtype=np.complex128)
        psi_for_fidelity=np.zeros(n_qubits,dtype=np.complex128)
        # for a in range(n_qubits):
        #     a_mb=2**(a)
        #     final_state=Statevector(circuit_time_evolution).data
        #     single_particle_vector[a_mb]=final_state[a_mb]
        #     psi_for_fidelity[a]=final_state[a_mb]
        # energies_qiskit[n]=Statevector(single_particle_vector).expectation_value(hamiltonian_t)
    print(Statevector(circuit_time_evolution).expectation_value(hamiltonian_q))
    transpiled_circuit_time_evolution=transpile(circuit_time_evolution.decompose(), optimization_level=3,basis_gates=['cx','s','h','rz','x'])
    
    #backend = FakeMarrakesh()
    # backend = service.get_backend("ibm_brisbane")

    # service = QiskitRuntimeService()
    # backend = service.least_busy(
    #     operational=True, simulator=False, min_num_qubits=127
    # )

    # print(backend)
    backend=StatevectorSimulator()
    
    estimator = Estimator(mode=backend)
    options = estimator.options
    # Turn on gate twirling.
    # options.twirling.enable_gates = True
    # options.twirling.num_randomizations = 60
    # options.twirling.shots_per_randomization = 100
    # options.resilience.pec_mitigation = True
    # options.resilience.pec_mitigation = True
    # # #options.resilience.pec.max_overhead = 300
    # options.dynamical_decoupling.enable = True
    # options.dynamical_decoupling.sequence_type = "XpXm"

    # options.resilience.zne_mitigation = True
    # options.resilience.zne.noise_factors = (1, 2, 3,4,5,6)
    # options.resilience.zne.extrapolator = "exponential"
    # estimator.options.resilience.zne.amplifier = "pea"


    pm = generate_preset_pass_manager(backend=backend, optimization_level=3)



    transpiled_qc=pm.run(transpiled_circuit_time_evolution)
    layout = transpiled_qc.layout

    h_squared=(hamiltonian_q @ hamiltonian_q).simplify()
    observables = [
        hamiltonian_q
    ]
    observables = [
        [observable.apply_layout(layout) for observable in observable_set]
        for observable_set in observables
    ]


    estimator_pub = (transpiled_qc ,observables)

    transpiled_qc.draw()

    #print(f">>> gate twirling is turned on: {sampler.options.twirling.enable_gates}")


    job = estimator.run([estimator_pub])
    result = job.result()
    energy_estimation=(np.sum(result[0].data.evs))
    energies_clean.append(energy_estimation)
    print(f'energy at time {time[idx_t]} is {energy_estimation:.4f} \n')